In [1]:
import numpy as np
import cv2
import open3d as o3d
import yaml

# Read camera calibration parameters from r3live_config_calib.yaml
with open('/home/hong/catkin_ws/src/r3live/config/r3live_config_calib.yaml', 'r') as f:
    calib_data = yaml.safe_load(f)

camera_matrix = np.array(calib_data['r3live_vio']['camera_intrinsic']).reshape(3, 3)
dist_coeffs = np.array(calib_data['r3live_vio']['camera_dist_coeffs'])
R = np.array(calib_data['r3live_vio']['camera_ext_R']).reshape(3, 3)
T = np.array(calib_data['r3live_vio']['camera_ext_t']).reshape(3, 1)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-11-24 15:54:16,575 - utils - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2023-11-24 15:54:16,576 - utils - NumExpr defaulting to 8 threads.


In [8]:
# Read image and pointcloud data
import os
pointcloud = o3d.geometry.PointCloud()
for i in range(16):
    image = cv2.imread(f"../log/image/{i}.png")
    if i % 4 == 0:
        pointcloud += o3d.io.read_point_cloud(f"../log/lidar/{int(i/4)}.pcd")
image_undistorted = cv2.undistort(image, camera_matrix, dist_coeffs)

lidar_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
from open3d.web_visualizer import draw
draw([pointcloud, lidar_frame])
# o3d.visualization.draw_geometries([pointcloud, lidar_frame])

NameError: name 'o3d' is not defined

In [16]:
# Color pointcloud with image using extrinsic parameters
R_inv = R
T_inv = T
extrinsic = np.hstack((R_inv, T_inv))
extrinsic = np.vstack((extrinsic, np.array([0, 0, 0, 1])))
pointcloud.transform(extrinsic)

intrinsics = o3d.camera.PinholeCameraIntrinsic(camera_matrix.shape[1], camera_matrix.shape[0], camera_matrix[0, 0], camera_matrix[1, 1], camera_matrix[0, 2], camera_matrix[1, 2])
colors = []
for i in range(len(pointcloud.points)):
    pixel = intrinsics.project(np.asarray(pointcloud.points[i]).reshape(1, -1))
    pixel = np.round(pixel.squeeze()).astype(int)
    if pixel[0] >= 0 and pixel[0] < image_undistorted.shape[1] and pixel[1] >= 0 and pixel[1] < image_undistorted.shape[0]:
        colors.append(image_undistorted[pixel[1], pixel[0]].astype(float) / 255)
else:
        colors.append([0, 0, 0])
pointcloud.colors = o3d.utility.Vector3dVector(colors) 

AttributeError: 'open3d.cpu.pybind.camera.PinholeCameraIntrinsic' object has no attribute 'project'

In [35]:
import open3d as o3d
import numpy as np



# Create camera frame and scanner frame
camera_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=30)
scanner_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=20)

# Apply transformation to scanner frame
trans = np.matrix([[-1, 0, 0, 0],
                   [0, 0.954709, 0.297542, 120.97],
                   [0, 0.297542, -0.954709, 16.053],
                   [0, 0, 0, 1]])
scanner_frame.transform(trans**-1)

# Visualize the coordinate frames
o3d.visualization.draw_geometries([camera_frame, scanner_frame])



In [50]:
import open3d as o3d
import numpy as np

# Create the coordinate frames
tf_OW_TW = np.matrix([[1, 1.74846e-07, 0, 0],
                     [-1.74846e-07, 1, 0, 0],
                     [0, 0, 1, -55],
                     [0, 0, 0, 1]])

tf_TW_RW = np.matrix([[-1, 8.74228e-08, 0, 840],
                     [-8.74228e-08, -1, 0, 0],
                     [0, 0, 1, -140],
                     [0, 0, 0, 1]])

tf_RW_RE = np.matrix([[-4.37114e-08, 4.37114e-08, -1, 516.906],
                     [1, 5.01057e-15, -4.37114e-08, 8.6513e-07],
                     [3.09988e-15, -1, -4.37114e-08, 195.997],
                     [0, 0, 0, 1]])

tf_RE_SP = np.matrix([[1, 8.74228e-08, 8.74228e-08, 0],
                     [8.74228e-08, -1, 7.64274e-15, 0],
                     [8.74228e-08, 0, -1, -57],
                     [0, 0, 0, 1]])

tf_SP_MC = np.matrix([[-1, 0, 0, 0],
                     [0, 0.954709, 0.297542, 120.97],
                     [0, 0.297542, -0.954709, 16.053],
                     [0, 0, 0, 1]])

tf_OW_MC = np.matrix([[-3.93403e-07, -0.297542, 0.954709, 250.041],
                     [1, -2.00517e-07, 3.49573e-07, -2.38945e-05],
                     [8.74228e-08, 0.954709, 0.297542, 231.967],
                     [0, 0, 0, 1]])

coordinate_frames = [
    o3d.geometry.TriangleMesh.create_coordinate_frame(size=50),
    o3d.geometry.TriangleMesh.create_coordinate_frame(size=30),
    o3d.geometry.TriangleMesh.create_coordinate_frame(size=10),
    o3d.geometry.TriangleMesh.create_coordinate_frame(size=5),
    o3d.geometry.TriangleMesh.create_coordinate_frame(size=5),
    o3d.geometry.TriangleMesh.create_coordinate_frame(size=5)
]

# Apply transformations to the coordinate frames
coordinate_frames[0]
coordinate_frames[1].transform(tf_SP_MC)
coordinate_frames[2].transform(tf_RE_SP@tf_SP_MC)
coordinate_frames[3].transform(tf_RW_RE@tf_RE_SP@tf_SP_MC)
coordinate_frames[4].transform(tf_TW_RW@tf_RW_RE@tf_RE_SP@tf_SP_MC)
coordinate_frames[5].transform(tf_OW_TW@tf_TW_RW@tf_RW_RE@tf_RE_SP@tf_SP_MC)

# Visualize the coordinate frames
o3d.visualization.draw_geometries(coordinate_frames)
